In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow.compat.v1 as tf
import numpy as np
import pandas as pd
import tempfile
from sklearn.model_selection import train_test_split

In [ ]:
data = pd.DataFrame(pd.read_csv("//kaggle//input//car-evaluation-data-set//car_evaluation.csv"))


data.columns=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'rating']

data.head(100)

In [ ]:
print('buying data is : ' , data['buying'].unique())
print('maint data is : ' , data['maint'].unique())
print('doors data is : ' , data['doors'].unique())
print('persons data is : ' , data['persons'].unique())
print('lug_boot data is : ' , data['lug_boot'].unique())
print('safety data is : ' , data['safety'].unique())
print('rating data is : ' , data['rating'].unique())

y=data['rating'].replace(['unacc','acc','vgood','good'],[0,1,2,3])
print(y)

In [ ]:
def categ(datas):
    lst=[]
    for i in list(datas.columns):
        j =tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_hash_bucket(i,hash_bucket_size=1000))
        lst.append(j)
    return lst

lstx=categ(data)[:-1]

print(lstx)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:,:-1], y, random_state=42)

print('X_train shape is ' , X_train.shape)
print('X_test shape is ' , X_test.shape)
print('y_train shape is ' , y_train.shape)
print('y_test shape is ' , y_test.shape)

In [ ]:
#-------------------DNN Classifier---------------
model = tf.estimator.DNNClassifier(
    feature_columns=lstx,n_classes=4,
    hidden_units=[512, 256, 128],
    optimizer=tf.train.ProximalAdagradOptimizer(
      learning_rate=0.1,
      l1_regularization_strength=0.001))

In [ ]:
input_func=tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=100,num_epochs=None,shuffle=True)
model.train(input_fn=input_func,steps = 1200)

In [ ]:
#------------------Model Evaluation--------------
input_fun=tf.estimator.inputs.pandas_input_fn(x=X_test,y=y_test,batch_size=100,num_epochs=None,shuffle=True)
Test_accuracy_score = model.evaluate(input_fn=input_fun, steps=1)["accuracy"]
print("Test Accuracy: {0:2f}".format(Test_accuracy_score))

Train_accuracy_score = model.evaluate(input_fn=input_func, steps=1)["accuracy"]
print("Train Accuracy: {0:2f}".format(Train_accuracy_score))


In [ ]:
input_test=tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=len(X_test),shuffle=False)
predictions=list(model.predict(input_fn=input_test))
final_preds_test = []
for pred in predictions:
    final_preds_test.append(pred['class_ids'][0])
    
    
    
    
input_train=tf.estimator.inputs.pandas_input_fn(x=X_train,batch_size=len(X_train),shuffle=False)
predictions=list(model.predict(input_fn=input_train))
final_preds_train = []
for pred in predictions:
    final_preds_train.append(pred['class_ids'][0])
    

In [ ]:
#-------------Model QC-----------------
import sklearn.metrics as metrs
import seaborn as sea
print(metrs.accuracy_score(y_test,final_preds_test))
print(metrs.accuracy_score(y_train,final_preds_train))

cf=metrs.confusion_matrix(y_test,final_preds_test)
sea.heatmap(cf,annot=True,fmt='d',cmap='gist_rainbow')
